In [34]:
# Necessary to parse date from string in french format to datetime objects
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

import os 
import glob
import urllib
import requests
import time
import json
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [38]:
# 1) First you need to download the web pages of your dashboard you want to analyze in html format.
# Make sur to include all the pages that cover the month you want to analyze

# 2) Adjust the month number associated in the variable below:
month_to_analyze = 11
year_to_analyze = 2021

# 3) Params to adjust for the request, including the Bearer token provided by OCR
authorization = 'Bearer eyJ'

min_range = list(range(0,100,20))
max_range = [i+19 for i in min_range]
ranges = list(zip(min_range, max_range))

In [39]:
def request_ocr(authorization, lower_range, upper_range):

    headers = {
        'authority': 'api.openclassrooms.com',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
        'dnt': '1',
        'accept-language': 'fr',
        'sec-ch-ua-mobile': '?0',
        'authorization': authorization,
        'content-type': 'application/json',
        'accept': 'application/json;version=0.1',
        'x-requested-with': 'XMLHttpRequest',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'range': 'items={}-{}'.format(lower_range, upper_range), # change that from 20 to 39, and so on
        'origin': 'https://openclassrooms.com',
        'sec-fetch-site': 'same-site',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://openclassrooms.com/',
    }

    params = (
        ('actor', 'expert'),
        ('life-cycle-status', 'completed,marked student as absent'),
    )

    response = requests.get('https://api.openclassrooms.com/users/7755063/sessions', headers=headers, params=params)

    return response



def parse_ocr_response(response):
    events = json.loads(response.text)
    info = [dict(zip(('date', 'student', 'status', 'type', 'level'),
            (event['sessionDate'], event['recipient']['displayableName'], event['status'], event['type'], event['projectLevel'])))
            for event in events]
    return info

    

In [40]:
output = []
for r in ranges:

    response = request_ocr(authorization=authorization, lower_range=r[0], upper_range=r[1])
    output.extend(parse_ocr_response(response=response))


df = pd.DataFrame(output)

In [41]:
# Parse date to datetime and set as index
df['date'] = pd.to_datetime(df['date'])

# Set the date as index and sort index
df = df.set_index('date')
df = df.sort_index()

# Filter by date (based on the month of interest)
date_mask = (df.index.map(lambda x: x.month) == month_to_analyze) & (df.index.year == year_to_analyze)
df = df[date_mask]
display(df.head(3))

,level,status,student,type
date,,,,
2021-11-01 09:00:00,3,completed,Alizée Chasse,mentoring
2021-11-01 13:00:00,3,completed,Alizée Chasse,mentoring
2021-11-01 14:00:00,3,completed,Justine Jouvinier,mentoring


In [42]:
# Remove useless status:
# mask = sessions_df['status'].isin(['Annulée tardivement', 'Annulée', 'Annulée\n'])

# Final Summary Table
final_table = df.pivot_table(index=['type'], columns=['level', 'status'], values='student', aggfunc='count').fillna(0)
display(final_table)

level,1,2,3
status,completed,completed,completed
type,,,
mentoring,6.0,1.0,34.0
presentation,2.0,0.0,1.0


In [ ]:
# import numpy as np

# rate = np.array([30, 15, 35, 40 , 20]) 

# invoice_amount = final_table.sort_values(by='level',axis=1, ascending=True).values.dot(rate.T)
# print(invoice_amount.sum())